# Figure 6: Climate projections (GCM and SSP)

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31")

## Data

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins = basins[["basin_id", "geometry"]].set_index("basin_id")

data = pd.read_csv("MS2 Results/zenodo/dataset_future.csv", index_col = "basin_id")
area = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").area_RGI6
data['basin_name']= data['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
basins = pd.concat([basins, area, data], axis=1)
basins["area_RGI6_percent"] = basins["area_RGI6"]/basins.area_RGI6.sum()

# major catchments
basins_m = basins.dropna(subset = ['basin_name'])
basins_spread = basins[basins.PPc_spread.abs() <= 8]

In [ ]:
# weighted-means
df_wm = {'PPc': [(basins.PPc_ssp126 * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                 (basins.PPc_ssp245 * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                 (basins.PPc_ssp370 * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                 (basins.PPc_ssp585 * basins.area_RGI6).sum() / basins.area_RGI6.sum()], 
        'T2Mc': [(basins.T2Mc_ssp126 * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                 (basins.T2Mc_ssp245 * basins.area_RGI6).sum() / basins.area_RGI6.sum(), 
                 (basins.T2Mc_ssp370 * basins.area_RGI6).sum() / basins.area_RGI6.sum(),
                 (basins.T2Mc_ssp585 * basins.area_RGI6).sum() / basins.area_RGI6.sum()]}

df_wm = pd.DataFrame(data=df_wm, index=["SSP126", "SSP245", "SSP370", "SSP585"])

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)
geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords  = [-43.2, -45.95,  -46.9,  -47.55,   -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords  = [-71.2, -71.7,   -73.8,   -71.7,   -72.2,   -72.3,   -72.1, -72.8, -69.2]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.03, column_widths = [0.34, 0.34, 0.32], shared_xaxes = True,
                    subplot_titles = ["Precipitation change (ΔPP)","Temperature change (ΔT2M)","Changes by scenario"],
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scattergeo", "rowspan": 2}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

# colors
cl = px.colors.colorbrewer.RdYlBu

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)

# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PPc_ssp245'], 
                            colorscale = [(0., cl[2]),(0.75, "#ffe9ba"),(1, cl[9])], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = -15, zmax = 5, colorbar=dict(len=0.45, x=0.24, y= 0.77, title='ΔPP (%)', thickness=20)), row=1, col=1)

fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2Mc_ssp245'], 
                            colorscale=["#ffe9ba", cl[4],  cl[1]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = 1.2, zmax = 2, colorbar=dict(len=0.45, x=0.58, y= 0.77, title='ΔT2M (ºC)', dtick = 0.2, thickness=20)), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins_spread['geometry'].to_json()),  locations = basins_spread.index, z = basins_spread.PPc_spread, 
                           colorscale = ["rgba(213,213,213,0)", "rgba(213,213,213,0)"], marker_line_color ='black', showscale= False, marker_line_width=0.20), row=1, col=1)

fig.add_annotation(text="(b)", font=dict(size=16), x=0.36, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

# layout a) and b)
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7, color = 'black'), opacity = 0.7, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=10, color = "rgba(0,0,0,0.7)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)

# Precipitation ssp spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins.sort_values("PPc_ssp126").PPc_ssp126/100, x = basins.sort_values("PPc_ssp126").area_RGI6_percent.cumsum(), 
                         marker_color = cl[9],  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPc_ssp245").PPc_ssp245/100, x = basins.sort_values("PPc_ssp245").area_RGI6_percent.cumsum(), 
                         marker_color = cl[8],  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPc_ssp370").PPc_ssp370/100, x = basins.sort_values("PPc_ssp370").area_RGI6_percent.cumsum(), 
                         marker_color = cl[3],   showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("PPc_ssp585").PPc_ssp585/100, x = basins.sort_values("PPc_ssp585").area_RGI6_percent.cumsum(), 
                         marker_color = cl[1],  showlegend = False), row=1, col=3)

fig.update_xaxes(range = [-0.02,1.02], dtick = 0.25, row = 1, col = 3)
fig.update_yaxes(range = [-0.2,0.07], title_text="Precipitation change (%)", side = "right", title_standoff = 0, tickangle = 0, tickformat = ',.0%', row = 1, col = 3)
fig.add_annotation(text="(c)", font=dict(size=16), x=0.03, y=0.06, showarrow=False, row=1, col=3)
fig.add_annotation(text="SSP5-8.5 ({:.1f}%)".format(df_wm.PPc["SSP585"]), font=dict(size=13, color = cl[1]), x=0.55, y=-0.08, textangle=-23, showarrow=False, row=1, col=3)
fig.add_annotation(text="SSP3-7.0 ({:.1f}%)".format(df_wm.PPc["SSP370"]), font=dict(size=13, color = cl[3]), x=0.27, y=-0.065, textangle=-20, showarrow=False, row=1, col=3)
fig.add_annotation(text="SSP2-4.5 ({:.1f}%)".format(df_wm.PPc["SSP245"]), font=dict(size=13, color = cl[8]), x=0.25, y=-0.04, textangle=-20, showarrow=False, row=1, col=3)
fig.add_annotation(text="SSP1-2.6 ({:.1f}%)".format(df_wm.PPc["SSP126"]), font=dict(size=13, color = cl[9]), x=0.25, y=0.00, textangle=-10, showarrow=False, row=1, col=3)


# Temperature ssp spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins.sort_values("T2Mc_ssp126").T2Mc_ssp126, x = basins.sort_values("T2Mc_ssp126").area_RGI6_percent.cumsum(), 
                         marker_color = cl[9], showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2Mc_ssp245").T2Mc_ssp245, x = basins.sort_values("T2Mc_ssp245").area_RGI6_percent.cumsum(), 
                         marker_color = cl[8], showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2Mc_ssp370").T2Mc_ssp370, x = basins.sort_values("T2Mc_ssp370").area_RGI6_percent.cumsum(), 
                         marker_color = cl[3], showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins.sort_values("T2Mc_ssp585").T2Mc_ssp585, x = basins.sort_values("T2Mc_ssp585").area_RGI6_percent.cumsum(), 
                         marker_color = cl[1], showlegend = False), row=2, col=3)

fig.update_xaxes(range = [-0.02,1.02], title_text="Probability of exceedance (glacier area; %)", title_standoff = 0, dtick = 0.25, tickangle = 0, tickformat = ',.0%', row = 2, col = 3)
fig.update_yaxes(range = [0.8,3], title_text="Temperature change (°C)", side = "right", title_standoff = 15, tickangle = 0, row = 2, col = 3)
fig.add_annotation(text="(d)", font=dict(size=16), x=0.03, y=2.9, showarrow=False, row=2, col=3)
fig.add_annotation(text="SSP5-8.5 ({:.1f}°C)".format(df_wm.T2Mc["SSP585"]), font=dict(size=13, color = cl[1]), x=0.5, y=2.83, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="SSP3-7.0 ({:.1f}°C)".format(df_wm.T2Mc["SSP370"]), font=dict(size=13, color = cl[3]), x=0.5, y=2.33, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="SSP2-4.5 ({:.1f}°C)".format(df_wm.T2Mc["SSP245"]), font=dict(size=13, color = cl[8]), x=0.5, y=1.7, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="SSP1-2.6 ({:.1f}°C)".format(df_wm.T2Mc["SSP126"]), font=dict(size=13, color = cl[9]), x=0.5, y=1.1, textangle=-5, showarrow=False, row=2, col=3)

# layout c and d
fig.update_traces(line = dict(width = 2), opacity = 0.8, row = 1, col = 3)
fig.update_traces(line = dict(width = 2), opacity = 0.8, row = 2, col = 3)
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.5)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(barmode = "overlay",  template = "seaborn")
fig.update_layout(autosize = False, width = 900, height = 600, margin = dict(l=10, r=10, b=10, t=30, pad=0, autoexpand=True))
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_6_GCM_SSP.png", scale=4)

In [ ]:
"Under the SSP1-2.6 scenario, the {:.0f}% of the glacier area is projected to experience a decline in precipitation (Fig. 6c). This percentage increases to {:.0f}% under the SSP5-8.5 scenario.".format(
    basins[basins["PPc_ssp126"] < 0].area_RGI6.sum()*100 / basins.area_RGI6.sum(),
    basins[basins["PPc_ssp585"] < 0].area_RGI6.sum()*100 / basins.area_RGI6.sum())

In [ ]:
"For temperature, the glacier area-weighted mean warming varies from {:.2f} ºC in SSP1-2.6 to {:.2f} ºC in SSP5-8.5 (Fig. 6d).".format(
    df_wm.T2Mc["SSP126"], df_wm.T2Mc["SSP585"])

In [ ]:
"For precipitation, the relative differences in terms of glacier area-weighted mean varies from {:.1f} % in SSP1-2.6 to {:.1f} % in SSP5-8.5.".format(
    df_wm.PPc["SSP126"], df_wm.PPc["SSP585"])